<a href="https://colab.research.google.com/github/juanmggb/maestria/blob/main/data/generate_data_parm_est.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18184 sha256=9282c744a1764231fb0590ed510a9ddd194115d9cab97ac097e0a7eb7d5fee56
  Stored in directory: /root/.cache/pip/wheels/00/3e/fa/5705bf59c2053c17c4799c3ab66a2e356c32f40a3044fe2134
Successfully built pyDOE


In [3]:
# Import libraries
import numpy as np 
import pandas as pd  
from scipy.integrate import odeint

In [6]:
!git clone https://github.com/juanmggb/maestria.git

Cloning into 'maestria'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 323 (delta 53), reused 18 (delta 6), pack-reused 203
Receiving objects: 100% (323/323), 9.31 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [7]:
cd maestria

/content/maestria


In [10]:
import utilis.functions as fn
from utilis.models import monod_model, inhibition_model

# Generate data

In [4]:
# Definir parametros 
mu_max = 1.2 # g/L
yxs = 0.2 
ks = 280 # g/L
ypx = 4 
ki = 0.3 # L/g

# Definir condiciones iniciales 
x0 = np.array([0.2, 40, 0])
# Definir tiempo de integracion 
tf = 60 
dt = 1 
time = np.arange(0, tf+dt, dt)

In [9]:
# Solve ODEs
x_m, s_m, p_m = odeint(monod_model, x0, time, args =(0, mu_max, yxs, ks, ypx,)).T
x_i, s_i, p_i = odeint(inhibition_model, x0, time, args =(0, mu_max, yxs, ks, ypx, ki, )).T

In [11]:
print(fn.final_conc(x_m, s_m, p_m))
print(fn.final_conc(x_i, s_i, p_i))

(8.185126166432566, 0.07436916783716456, 31.94050466573027)
(6.6386536122346005, 7.806731938826973, 25.754614448938405)


In [12]:
predicted_monod = fn.create_df(time, x_m, s_m, p_m)
predicted_monod.head()

,t,x,s,p
0,0,0.200000,40.000000,0.000000
1,1,0.232307,39.838466,0.129227
2,2,0.269678,39.651610,0.278712
3,3,0.312854,39.435728,0.451417
4,4,0.362666,39.186670,0.650664


In [13]:
predicted_inhibition = fn.create_df(time, x_i, s_i, p_i)
predicted_inhibition.head()

,t,x,s,p
0,0,0.200000,40.000000,0.000000
1,1,0.212370,39.938151,0.049479
2,2,0.225510,39.872450,0.102040
3,3,0.239469,39.802654,0.157877
4,4,0.254299,39.728504,0.217197


In [14]:
predicted_noise_m = fn.add_noise(predicted_monod, 0.1, 1, 1)
predicted_noise_i = fn.add_noise(predicted_inhibition, 0.1, 1, 1)

In [15]:
predicted_noise_m.to_csv('data/data_monod.csv', index=False)
predicted_noise_i.to_csv('data/data_inhibition.csv', index=False)